# **CASE: COMPRA DE PERFUMES**

---

### Instalação de bibliotecas

In [ ]:
# Para cálculo do VIF
install.packages("rms")
# Para cálculo do KS e AUC
install.packages("ROCR")

### Carregamento de bibliotecas

In [ ]:
library(rms)
library(ROCR)

### Leitura da base de dados

In [ ]:
dados_perfume <- read.table(file = "Compra_Perfumes.txt",
                            sep = "\t",
                            dec = ",",
                            header = TRUE)

### Visualização da base de dados

In [ ]:
View(dados_perfume)

### Análise exploratória breve

In [ ]:
# Medidas resumo: idade
summary(dados_perfume$Idade)

In [ ]:
# Tabela de frequências: gênero
table(dados_perfume$Genero)
prop.table(table(dados_perfume$Genero))

In [ ]:
# Tabela de frequências: cidade
table(dados_perfume$Cidade)
prop.table(table(dados_perfume$Cidade))

In [ ]:
# Tabela de frequências: sistema operacional
table(dados_perfume$Sistema_Operac)
prop.table(table(dados_perfume$Sistema_Operac))

In [ ]:
# Tabela de frequências: utilização do cupom
table(dados_perfume$Utilizou_Cupom)
prop.table(table(dados_perfume$Utilizou_Cupom))

### Análise bivariada: variáveis resposta vs. variáveis explicativas

In [ ]:
# Gráfico de boxplots: idade versus resposta
boxplot(Idade ~ Utilizou_Cupom, data = dados_perfume, col = "darkturquoise", main = "Idade vs. Utilização do Cupom")

In [ ]:
# Tabela de frequências: gênero versus resposta (somando 100% nas linhas)
table(dados_perfume$Genero, dados_perfume$Utilizou_Cupom)
prop.table(table(dados_perfume$Genero, dados_perfume$Utilizou_Cupom), 1)

In [ ]:
# Tabela de frequências: cidade versus resposta (somando 100% nas linhas)
table(dados_perfume$Cidade, dados_perfume$Utilizou_Cupom)
prop.table(table(dados_perfume$Cidade, dados_perfume$Utilizou_Cupom), 1)

In [ ]:
# Tabela de frequências: sistema operacional versus resposta (somando 100% nas linhas)
table(dados_perfume$Sistema_Operac, dados_perfume$Utilizou_Cupom)
prop.table(table(dados_perfume$Sistema_Operac, dados_perfume$Utilizou_Cupom), 1)

### Modelo de regressão logística múltipla

In [ ]:
# Comando auxiliar para para omitir notação científica nos p-valores e controlar largura dos outputs na tela do Colab
options(scipen = 999, width = 200)

In [ ]:
# Ajuste do modelo 1: inicial
regressao_1 <- glm(Utilizou_Cupom ~
                     Genero +
                     Idade +
                     Cidade +
                     Sistema_Operac,
                   family = binomial(link = 'logit'),
                   data = dados_perfume)

summary(regressao_1)

In [ ]:
# Ajuste do modelo 2: retirando Sistema_Operac
regressao_2 <- glm(Utilizou_Cupom ~
                     Genero +
                     Idade +
                     Cidade,
                   family = binomial(link = 'logit'),
                   data = dados_perfume)

summary(regressao_2)

In [ ]:
# Avaliação de colinearidade no modelo 2
data.frame(VIF = vif(regressao_2))

### Intervalos de confiança

In [ ]:
# Escolha do beta (0 para intercepto, ou 1, 2, ... para os parâmetros referentes a cada variável explicativa)
beta = 0

In [ ]:
# Cálculo do intervalo
print("Limite inferior")
as.numeric(regressao_2$coefficients[beta + 1] - 1.96 * coef(summary(regressao_2))[beta + 1, "Std. Error"])
print("Limite superior")
as.numeric(regressao_2$coefficients[beta + 1] + 1.96 * coef(summary(regressao_2))[beta + 1, "Std. Error"])

### Aplicação do modelo e definição de resposta predita

In [ ]:
# Aplicação do modelo na base (criação de uma nova coluna chamada "probabilidade")
dados_perfume$probabilidade <- predict(regressao_2,
                                       dados_perfume,
                                       type = "response")

In [ ]:
# Definição de ponto de corte (padrão: % de 1's na amostra)
ponto_corte <- mean(dados_perfume$Utilizou_Cupom)
ponto_corte

In [ ]:
# Definição da resposta predita pelo modelo (criação de uma nova coluna chamada "predito")
dados_perfume$predito <- as.factor(ifelse(dados_perfume$probabilidade > ponto_corte, 1, 0))

### Análise de desempenho

In [ ]:
# Tabela de classificação
tabela <- table(dados_perfume$Utilizou_Cupom, dados_perfume$predito)

In [ ]:
# Acurácia
(tabela[1,1] + tabela[2,2]) / sum(tabela)

In [ ]:
# Especificidade
tabela[1,1] / (tabela[1,1] + tabela[1,2])

In [ ]:
# Sensibilidade
tabela[2,2] / (tabela[2,1] + tabela[2,2])

In [ ]:
# KS
pred <- prediction(dados_perfume$probabilidade, dados_perfume$Utilizou_Cupom)
perf <- performance(pred, "tpr", "fpr")
ks <- max(attr(perf, 'y.values')[[1]] - attr(perf, 'x.values')[[1]])

print(ks)

In [ ]:
# AUC
pred <- prediction(dados_perfume$probabilidade, dados_perfume$Utilizou_Cupom)
auc <- performance(pred, "auc")
auc <- auc@y.values[[1]]

print(auc)

### Exemplo de uso (aplicação) do modelo

In [ ]:
# Criação de base de dados com um novo cliente, do gênero feminino, com 50 anos de idade e que mora na cidade de São Paulo
# Obs.: os nomes das colunas e padrão de conteúdo devem ser idênticos aos da base utilizada para construção do modelo
novos_dados <- data.frame(Genero = c("Feminino"),
                          Idade  = c(50),
                          Cidade = c("Sao_Paulo"))

In [ ]:
# Aplicação do modelo
novos_dados$PROB_USAR_CUPOM <- predict(regressao_2, novos_dados, type = "response")
View(novos_dados)